# 1. Imports

In [32]:
from pathlib import Path
from dataclasses import dataclass

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import seaborn as sns
from skimage import io

plt.rcParams.update({
    'axes.unicode_minus': True,  # Correctly display minus sign
    'figure.dpi': 300,            # Image resolution
    'axes.titlesize': 20,         # Title font size
    'axes.titleweight': 'bold', # Title font weight
    'axes.labelsize': 18,         # Axis label font size
    'axes.labelweight': 'bold',  # Axis label font weight
    'font.family': 'Arial',       # Set global font to Arial
    'font.size': 16,              # Base font size
    'legend.fontsize': 16,        # Legend font size
    'axes.spines.top': False,      # Remove top spine
    'axes.spines.right': False,     # Remove right spine
    'axes.linewidth': 3,   # Axis line width
    'lines.linewidth': 3,  # Line width
    'lines.solid_joinstyle': 'round',  # Line join style
    'lines.solid_capstyle': 'round',    # Line cap style
    'image.interpolation': 'nearest',  # Image interpolation
    'pdf.compression': 9  # PDF compression level (0-9)
})

# 2. Configuration

In [42]:
@dataclass
class config:
    verification_result_file: Path = Path("../results/all_combined_all_rounds_crop_summary_manual_annotated_with_genotyping_20260129_secondary_modification.xlsx")
    colony_size_result_file: Path = Path("../results/nonE_strain_or_commented_E_genotyping_results.xlsx")

    def __post_init__(self):
        self.verification_result: pd.DataFrame = pd.read_excel(self.verification_result_file).query("Kept == 'YES'").copy()
        self.genotyping_summary = self.verification_result.query("Genotyping == 'YES' ").set_index(["gene_num", "gene_name", "round", "colony_id","date"])
        self.colony_size_result: pd.DataFrame = pd.read_excel(self.colony_size_result_file, index_col=[1,2,0,3,4])

# 3. Functions

In [ ]:
def verification_summary(sub_df: pd.DataFrame):

    round_gene_num = sub_df[['round', 'gene_num']].drop_duplicates().apply(lambda row: f"{row['round']}_{row['gene_num']}", axis=1).tolist()
    round_gene_num = ",".join(round_gene_num)

    kept_df = sub_df.query('Kept == "YES"')
    verification_phenotype = kept_df['verification_phenotype'].unique().tolist()
    if len(verification_phenotype) > 1:
        display(kept_df)
        raise ValueError("Multiple verification phenotypes found in group.")
    elif len(verification_phenotype) == 0:
        verification_phenotype = np.nan
    else:
        verification_phenotype = verification_phenotype[0]
    
    verification_essentiality = kept_df['verification_essentiality'].unique().tolist()
    if len(verification_essentiality) > 1:
        display(kept_df)
        raise ValueError("Multiple verification essentialities found in group.")
    elif len(verification_essentiality) == 0:
        verification_essentiality = np.nan
    else:
        verification_essentiality = verification_essentiality[0]
    support_colony_count = kept_df.shape[0]
    combined_comments = kept_df[kept_df["Comments"].notna()].apply(lambda row: f"{row['round']}_{row["gene_num"]}_{row['colony_id']}: {row['Comments'] if pd.notna(row['Comments']) else ''}", axis=1).tolist()
    combined_comments = ";".join(combined_comments) if len(combined_comments) > 0 else ''

    return pd.Series({
        "round_gene_num": round_gene_num,
        "verification_phenotype": verification_phenotype,
        "verification_essentiality": verification_essentiality,
        # "support_colony_count": support_colony_count,
        "comments": combined_comments
    })

In [43]:
# %% ------------------------------------ Functions ------------------------------------ %%
def tetrad_plate_plot(genes, cfg, plate_genotyping_statistics_summary, filtered_genotyping_results, return_fig=False):
    # gene_index = cfg.genotyping_summary[cfg.genotyping_summary.index.get_level_values("gene_name").isin(genes)].index
    index_names = ['gene_num', 'gene_name', 'round', 'colony_id', 'date']
    verification_result = cfg.verification_result.set_index(index_names)
    gene_index = cfg.verification_result.query("gene_name in @genes")[index_names]
    # Sort by gene order in the list
    # selected_colony = gene_index.to_frame().assign(
    selected_colony = gene_index.assign(
        gene_order=lambda df: df['gene_name'].map({gene: i for i, gene in enumerate(genes)})
    ).sort_values('gene_order').drop('gene_order', axis=1).set_index(index_names).index
    sorted_colony_size_result = cfg.colony_size_result.sort_index()
    filtered_genotyping_results = filtered_genotyping_results.sort_index()
    fig, axes = plt.subplots(len(selected_colony), 6, figsize=(30, 3 * len(selected_colony)))
    for row, colony in enumerate(selected_colony):
        for col, col_image in enumerate(["3d", "4d", "5d", "6d", "HYG"]):
            image_path = verification_result.loc[colony, f"{col_image}_image_path"]
            if col_image != "HYG":
                try:
                    image = io.imread(image_path)
                    axes[row, col].imshow(image, cmap="gray")
                except:
                    axes[row, col].axis("off")
                    continue
                try:
                    median_area_fraction = plate_genotyping_statistics_summary.loc[colony, f"median_area_day{col+3}"]
                    mean_area_fraction = plate_genotyping_statistics_summary.loc[colony, f"mean_area_day{col+3}"]
                    deletion_with_area_fraction = plate_genotyping_statistics_summary.loc[colony, "deletion_with_area_fraction"]
                    axes[row, col].set_title(f"area fraction (deletion/WT):\nmedian({median_area_fraction}) mean({mean_area_fraction})\n detectable deletion fraction (day6): {deletion_with_area_fraction}")
                    day=col + 3
                    colony_coordinates = sorted_colony_size_result.loc[colony]
                    for idx, region in colony_coordinates.iterrows():
                        cx, cy = region[f"grid_point_x_day{day}"], region[f"grid_point_y_day{day}"]
                        genotype = region["genotype"]
                        color = 'green' if genotype == "WT" else 'red'
                        area = region[f"area_day{day}"]
                        if area != 0:
                            circle = Circle((cx, cy), 30, edgecolor=color, facecolor='none', linewidth=2, alpha=0.4)
                        else:
                            circle = Circle((cx, cy), 30, edgecolor=color, facecolor='none', linewidth=2, alpha=0.4, linestyle="--")
                        axes[row, col].add_patch(circle)
                except KeyError:
                    pass
            else:
                try:
                    image = io.imread(image_path)
                    axes[row, col].imshow(image)
                except:
                    pass
                axes[row, col].set_title("_".join(map(str, colony)) + "\nHYG Plate")
            
            axes[row, col].axis("off")
        try:
            ax_area = axes[row, -1]
            colony_regions = filtered_genotyping_results.loc[colony]
            area_stat = colony_regions.set_index("genotype", append=True).filter(like="area_day")
            area_stat["area_day0"] = 0
            area_stat = area_stat.rename_axis("day", axis=1).stack().reset_index().rename(columns={0: "area"})
            area_stat["day_num"] = area_stat["day"].str.extract(r'day(\d+)').astype(int)
            last_day = area_stat["day_num"].max()
            # area_stat = area_stat.groupby(["col", "day"]).filter(lambda x: x.query("genotype == 'WT'").shape[0]/x.shape[0] == 0.5)
            last_day_WT_colonies_area_mean = area_stat.query("genotype == 'WT' and day_num == @last_day")["area"].median()
            area_stat["area[normalized]"] = area_stat["area"] / last_day_WT_colonies_area_mean
            area_stat = area_stat.query("genotype in ['WT', 'Deletion']")
            sns.lineplot(x="day_num", y="area[normalized]", hue="genotype", data=area_stat, ax=ax_area, palette={"WT": "green", "Deletion": "red"}, errorbar=("pi", 50), estimator="median")
            WT_count = colony_regions.query("genotype == 'WT'")["genotype"].count()
            deletion_count = colony_regions.query("genotype == 'Deletion'")["genotype"].count()
            ax_area.set_title(f"Colony Area Over Time:\n WT ({WT_count}) vs Deletion ({deletion_count})")
            ax_area.axhline(1, color='gray', linestyle='--')
        except KeyError:
            axes[row, -1].axis("off")
    plt.tight_layout()
    if return_fig:
        return fig
    else:
        plt.show()
        plt.close()            


# 4. Load data

In [44]:
def genotyping_statistics(sub_df: pd.DataFrame):
    total_colonies = sub_df.shape[0]
    filtered_colonies = sub_df.groupby(["gene_num", "gene_name", "round", "colony_id","date", "col"]).filter(lambda x: x.query("genotype == 'WT'").shape[0]/x.shape[0] == 0.5)
    filtered_count = filtered_colonies.shape[0]
    filtering_fraction = filtered_count / total_colonies if total_colonies != 0 else 0
    deletion_with_area = filtered_colonies.query("genotype == 'Deletion' and area_day6 > 0")
    deletion_with_area_count = deletion_with_area.shape[0]
    deletion_without_area = filtered_colonies.query("genotype == 'Deletion' and area_day6 == 0")
    deletion_without_area_count = deletion_without_area.shape[0]
    deletion_with_area_fraction = deletion_with_area_count / filtered_colonies.query("genotype == 'Deletion'").shape[0] if filtered_colonies.query("genotype == 'Deletion'").shape[0] != 0 else 0

    sub_area_table = filtered_colonies.set_index("genotype").filter(like="area_day").fillna(0)
    median_fraction = sub_area_table.groupby(by=sub_area_table.index.names).median().T
    median_fraction_per_day = (median_fraction["Deletion"] / median_fraction["WT"]).round(3).sort_index()
    mean_fraction = sub_area_table.groupby(by=sub_area_table.index.names).mean().T
    mean_fraction_per_day = (mean_fraction["Deletion"] / mean_fraction["WT"]).round(3).sort_index()

    stats = {
        "total_colonies": int(total_colonies),
        "filtered_count": int(filtered_count),
        "filtering_fraction": round(filtering_fraction, 2),
        "deletion_with_area_count": int(deletion_with_area_count),
        "deletion_without_area_count": int(deletion_without_area_count),
        "deletion_with_area_fraction": round(deletion_with_area_fraction, 2),
    }
    stats.update(median_fraction_per_day.add_prefix("median_").to_dict())
    stats.update(mean_fraction_per_day.add_prefix("mean_").to_dict())

    
    # if deletion_with_area_count == 0:
    #     detectable_deletion_colony_median_area_fraction = median_fraction_per_day.copy()
    #     detectable_deletion_colony_median_area_fraction = np.nan
    #     detectable_deletion_colony_median_area_fraction = detectable_deletion_colony_median_area_fraction.add_prefix("detectable_deletion_median_").to_dict()
    #     detectable_deletion_colony_mean_area_fraction = mean_fraction_per_day.copy()
    #     detectable_deletion_colony_mean_area_fraction = np.nan
    #     detectable_deletion_colony_mean_area_fraction = detectable_deletion_colony_mean_area_fraction.add_prefix("detectable_deletion_mean_").to_dict()
    #     stats.update(detectable_deletion_colony_median_area_fraction)
    #     stats.update(detectable_deletion_colony_mean_area_fraction)
    # else:
    #     with_area_colonies = filtered_colonies.query("area_day6 > 0")
    #     with_area_table = with_area_colonies.set_index("genotype").filter(like="area_day").fillna(0)
    #     median_fraction_with_area = with_area_table.groupby(by=with_area_table.index.names).median().T
    #     detectable_deletion_colony_median_area_fraction = (median_fraction_with_area["Deletion"] / median_fraction_with_area["WT"]).round(3).sort_index().add_prefix("detectable_deletion_median_")
    #     mean_fraction_with_area = with_area_table.groupby(by=with_area_table.index.names).mean().T
    #     detectable_deletion_colony_mean_area_fraction = (mean_fraction_with_area["Deletion"] / mean_fraction_with_area["WT"]).round(3).sort_index().add_prefix("detectable_deletion_mean_")
    #     stats.update(detectable_deletion_colony_median_area_fraction.to_dict())
    #     stats.update(detectable_deletion_colony_mean_area_fraction.to_dict())

    return pd.Series(stats)

In [45]:
# %% ------------------------------------ Main Function ------------------------------------ %%
cfg = config()
index = cfg.genotyping_summary.index
filtered_genotyping_results = cfg.colony_size_result.loc[index].groupby(["gene_num", "gene_name", "round", "colony_id","date", "col"]).filter(lambda x: x.query("genotype == 'WT'").shape[0]/x.shape[0] == 0.5)

plate_genotyping_statistics_summary = cfg.colony_size_result.loc[index].groupby(["gene_num", "gene_name", "round", "colony_id","date"]).apply(genotyping_statistics)
plate_genotyping_statistics_summary = plate_genotyping_statistics_summary.astype(
    {
        "total_colonies": int,
        "filtered_count": int,
        "filtering_fraction": float,
        "deletion_with_area_count": int,
        "deletion_without_area_count": int,
        "deletion_with_area_fraction": float,
    }
)

gene_genotyping_statistics_summary = cfg.colony_size_result.loc[index].groupby(["gene_name"]).apply(genotyping_statistics)
gene_genotyping_statistics_summary = gene_genotyping_statistics_summary.astype(
    {
        "total_colonies": int,
        "filtered_count": int,
        "filtering_fraction": float,
        "deletion_with_area_count": int,
        "deletion_without_area_count": int,
        "deletion_with_area_fraction": float,
    }
)

# area_table = filtered_genotyping_results.set_index("genotype", append=True).filter(like="area_day").fillna(0)
# area_table_unstack = area_table.groupby(by=area_table.index.names).median().unstack(level="genotype").reorder_levels([1,0], axis=1)
# relative_fraction_per_day = (area_table_unstack["Deletion"] / area_table_unstack["WT"]).round(3).sort_index()
# area_fraction = area_table.groupby("gene_name").apply(calculate_area_fraction).droplevel(-1).reset_index()

# 5. Summary

In [46]:
version = "20260129modified"

In [55]:
# summary = cfg.verification_result.query("Kept =='YES'").drop_duplicates(subset=["systematic_id", "gene_name", "verification_phenotype", "verification_essentiality"])[["systematic_id", "gene_name", "verification_phenotype", "verification_essentiality", "Comments"]]
summary = cfg.verification_result.query("Kept =='YES'").groupby(["systematic_id", "gene_name"]).apply(verification_summary).reset_index()[["systematic_id", "gene_name", "verification_phenotype", "verification_essentiality", "round_gene_num", "comments"]]
summary["gene_name"].value_counts()
summary_with_area_fraction = summary.merge(gene_genotyping_statistics_summary.reset_index(), on="gene_name", how="left").copy()
summary_with_area_fraction.to_csv(f"../results/{version}_verification_summary.csv", index=False)

/tmp/ipykernel_552749/4221866704.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary = cfg.verification_result.query("Kept =='YES'").groupby(["systematic_id", "gene_name"]).apply(verification_summary).reset_index()[["systematic_id", "gene_name", "verification_phenotype", "verification_essentiality", "round_gene_num", "comments"]]


# 6. Analysis

## E genes without any comment

重新检查发现轻微可见克隆的基因：
vrs2
SPAC30C2.03
ups1
pat1
mrps26
rxt3
mrps9
mcs6
mrpl3

In [ ]:
E_genes_without_comments = summary_with_area_fraction.query("verification_phenotype == 'E' and Comments.isna()").sort_values("mean_area_day6")["gene_name"].tolist()
fig = tetrad_plate_plot(E_genes_without_comments, cfg, plate_genotyping_statistics_summary, filtered_genotyping_results, return_fig=True)
plt.savefig(f"../results/{version}_tetrad_plate_plots_E_genes_without_comments.pdf", bbox_inches='tight', dpi=300)
plt.close()

In [ ]:
recheck_with_tiny_colonies_E_genes = [
    "vrs2",
    "SPAC30C2.03",
    "ups1",
    "pat1",
    "mrps26",
    "rxt3",
    "mrps9",
    "mcs6",
    "mrpl3",
]
fig = tetrad_plate_plot(recheck_with_tiny_colonies_E_genes, cfg, plate_genotyping_statistics_summary, filtered_genotyping_results, return_fig=True)
plt.savefig(f"../results/{version}_recheck_with_tiny_colonies_E_genes.pdf", bbox_inches='tight', dpi=300)
plt.close()

## E genes with comments

In [ ]:
E_genes_with_comments = summary_with_area_fraction.query("verification_phenotype == 'E' and Comments.notna()").sort_values("mean_area_day6")["gene_name"].tolist()
fig = tetrad_plate_plot(E_genes_with_comments, cfg, plate_genotyping_statistics_summary, filtered_genotyping_results, return_fig=True)
plt.savefig(f"../results/{version}_tetrad_plate_plots_E_genes_with_comments.pdf", bbox_inches='tight', dpi=300)
plt.close()

## E and tiny colonies

In [48]:
phenotype_order = ["E","E,tiny colonies","E,small colonies","E,WT","tiny colonies","small colonies","WT"]
summary_with_area_fraction["verification_phenotype"].value_counts().sort_index(
    key = lambda x: x.map({v: i for i, v in enumerate(phenotype_order)})
)

verification_phenotype
E                   170
E,small colonies     14
E,WT                  1
small colonies      177
WT                   48
Leu-condition         1
Name: count, dtype: int64

In [ ]:
E_and_tiny_colonies_genes = summary_with_area_fraction.query("verification_phenotype == 'E,tiny colonies'").sort_values("mean_area_day6")["gene_name"].tolist()
fig = tetrad_plate_plot(E_and_tiny_colonies_genes, cfg, plate_genotyping_statistics_summary, filtered_genotyping_results, return_fig=True)
plt.savefig(f"../results/{version}_tetrad_plate_plots_E_and_tiny_colonies_genes.pdf", bbox_inches='tight', dpi=300)
plt.close()

## E and small colonies

In [49]:
E_and_small_colonies_genes = summary_with_area_fraction.query("verification_phenotype == 'E,small colonies'").sort_values("mean_area_day6")["gene_name"].tolist()
fig = tetrad_plate_plot(E_and_small_colonies_genes, cfg, plate_genotyping_statistics_summary, filtered_genotyping_results, return_fig=True)
plt.savefig(f"../results/{version}_tetrad_plate_plots_E_and_small_colonies_genes.pdf", bbox_inches='tight', dpi=300)
plt.close()

## tiny colonies

In [ ]:
tiny_colonies_genes = summary_with_area_fraction.query("verification_phenotype == 'tiny colonies'").sort_values("mean_area_day6")["gene_name"].tolist()
fig = tetrad_plate_plot(tiny_colonies_genes, cfg, plate_genotyping_statistics_summary, filtered_genotyping_results, return_fig=True)
plt.savefig(f"../results/{version}_tetrad_plate_plots_tiny_colonies_genes.pdf", bbox_inches='tight', dpi=300)
plt.close()

## small colonies with small colonies

In [50]:
top_small_colonies_genes = summary_with_area_fraction.query("verification_phenotype == 'small colonies'").sort_values("mean_area_day6")["gene_name"].tolist()[:50]
fig = tetrad_plate_plot(top_small_colonies_genes, cfg, plate_genotyping_statistics_summary, filtered_genotyping_results, return_fig=True)
plt.savefig(f"../results/{version}_tetrad_plate_plots_top_small_colonies_genes.pdf", bbox_inches='tight', dpi=300)
plt.close()